# Conditional Expected Drawdown Demo

#### Start by importing modules and the CED function

In [5]:
import pandas as pd
import numpy as np 
import yfinance as yf

from conditional_drawdown import drawdown as dd